<a href="https://colab.research.google.com/github/ghedin-alison/NLP/blob/main/Template_Trabalho_Final_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Participantes (RM - NOME):***<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>

###**Criar um classificador de chamados aplicando técnicas de PLN**
---

A **DinDinAgora** tem um canal de atendimento via chat e precisar classificar os assuntos dos atendimentos para melhorar as tratativas dos chamados dos clientes. O canal recebe textos abertos dos clientes relatando o problema e/ou dúvida e depois é direcionado para algum uma área especialista no assunto para uma melhor tratativa.​

Crie um modelo classificador de assuntos aplicando técnicas de PLN, que consiga classificar através de um texto o assunto conforme disponível na base de dados [1] para treinamento e validação do modelo seu modelo.​

O modelo precisar atingir um score na **métrica F1 Score superior a 75%**. Utilize o dataset [1] para treinar e testar o modelo, separe o dataset em duas amostras (75% para treinamento e 25% para teste com o randon_state igual a 42).​

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

**Composição da nota:​**

**50%** - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, organização do pipeline, etc.)​

**50%** - Baseado na performance (score) obtida com a amostra de teste no pipeline do modelo campeão (validar com  a Métrica F1 Score). **Separar o pipeline completo do modelo campeão conforme template.​**

**[1] = ​https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv**

Obs.: Para a métrica F1 Score, usar o parâmetro average = 'weighted'.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

In [1]:
# instalações de pacotes
!pip install spacy
!python -m spacy download pt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-03-19 21:51:58.805025: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 21:52:00.468766: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-19 21:52:00.468924: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/

In [2]:
#import de bibliotecas
import spacy
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import spacy
import nltk



In [3]:
# CARREGANDO O DATA FRAME
df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')

# Façam o download do arquivo e utilizem localmente durante os testes

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21072 entries, 0 to 21071
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_reclamacao         21072 non-null  int64 
 1   data_abertura         21072 non-null  object
 2   categoria             21072 non-null  object
 3   descricao_reclamacao  21072 non-null  object
dtypes: int64(1), object(3)
memory usage: 658.6+ KB


Bom desenvolvimento!

In [26]:
df.head()

,id_reclamacao,data_abertura,categoria,descricao_reclamacao,text_lemma,text_lemma_verbos
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,"Bom dia, meu nome é xxxx xxxx e agradeço se vo...","bom dia , meu nome ser xxxx xxxx e agradeço se...","Bom dia , meu nome é xxxx xxxx e agradeço se v..."
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...,Atualizei meu cartão xxxx xxxx em xx / xx/2018...,Atualizei meu cartão xxxx xxxx em xx / xx/2018...
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,O cartão Chase foi relatado em xx/xx/2019. No ...,o cartão Chase ser relatar em xx / xx/2019 . e...,O cartão Chase foi relatar em xx / xx/2019 . N...
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,"Em xx/xx/2018, enquanto tentava reservar um ti...","em xx / xx/2018 , enquanto tentar reservar um ...","Em xx / xx/2018 , enquanto tentar reservar um ..."
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,"Meu neto me dê cheque por {$ 1600,00} Eu depos...","meu neto eu dê cheque por { $ 1600,00 } Eu dep...","Meu neto me dê cheque por { $ 1600,00 } Eu dep..."


In [5]:
df.shape

(21072, 4)

In [6]:
# aqui é melhor equilibrar a base de dados
df.categoria.value_counts()

Serviços de conta bancária             5161
Cartão de crédito / Cartão pré-pago    5006
Roubo / Relatório de disputa           4822
Hipotecas / Empréstimos                3850
Outros                                 2233
Name: categoria, dtype: int64

In [7]:
nlp = spacy.load("pt_core_news_sm")

In [8]:
# função de lematização completa do documento
def lematiza_texto(texto: str):
  sent = []
  doc = nlp(texto)
  for word in doc:
    sent.append(word.lemma_)
  return " ".join(sent)

# função de lematização de verbos do documento
def lematiza_verbos(texto:str):
  sent = []
  doc = nlp(texto)
  for word in doc:
    if word.pos_ == "VERB":
      sent.append(word.lemma_)
    else:
      sent.append(word.text)
  return " ".join(sent)


In [9]:
# aplica a lematização de no dataframe criando novas colunas
df["text_lemma"] = df.descricao_reclamacao.apply(lematiza_texto)

In [10]:
# aplica a lematização de no dataframe criando novas colunas
df["text_lemma_verbos"] = df.descricao_reclamacao.apply(lematiza_verbos)

In [16]:
# divisão das amostras em treino e teste
df_treino, df_teste = train_test_split(df, 
                                       test_size=0.3, 
                                       random_state=42)

In [24]:
# Regressão Logistica com lematização em verbos

nltk.download('stopwords')
nlp = spacy.load('pt_core_news_sm')

#stopwords combinadas
stops_spacy = nlp.Defaults.stop_words
stops_nltk = nltk.corpus.stopwords.words('portuguese')
stops = list(set(stops_spacy).union(set(stops_nltk)))
lista = ["pouca", "demais", "dez", "mal", "não", "porém", "nenhuma", "nada", "sim", "menos", "boa", "bom"]

print(f" stops antes da atualização{len(stops)}")

for i in lista:
  stops.remove(i)

print(f" stops antes da atualização{len(stops)}")

# vetorização com Tfid
vetor_tfid = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops, norm='l2')
vetor_tfid.fit(df_treino.text_lemma)
text_vect_treino = vetor_tfid.transform(df_treino.text_lemma)

#treina modelo
linear_model = LogisticRegression(random_state=42, class_weight='balanced')
linear_model.fit(text_vect_treino, df_treino["categoria"])# aqui eu treino no modelo Decision tree associando minhas features e targets

# testa e verifica a predição
text_vect_test = vetor_tfid.transform(df_teste.text_lemma)
pred = linear_model.predict(text_vect_test)

# mede acuracia
accuracy = accuracy_score(pred, df_teste['categoria'])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


 stops antes da atualização500
 stops antes da atualização488


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [25]:
f1_score(df_teste['categoria'], pred, average='weighted')

0.9122010027263365

####**Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.